# **Desafio Técnico - PicPay**

---

## **Instalação de pacotes e download dos dados**

In [ ]:
!wget  https://github.com/PicPay/case-machine-learning-engineer-pleno/raw/refs/heads/main/notebook/airports-database.zip
!unzip ./airports-database.zip

---

## **Preparação dos dados e funções auxiliares**

In [2]:
import sys
import pandas as pd
import numpy as np
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from google.colab import userdata

In [3]:
spark = SparkSession.builder.appName("picpay").getOrCreate()

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType

schema = StructType([
    StructField("id", IntegerType(), False),  # Um identificador único para cada registro de voo.
    StructField("year", IntegerType(), False),  # O ano em que o voo ocorreu (2013 neste conjunto de dados).
    StructField("month", IntegerType(), False),  # O mês em que o voo ocorreu (1 a 12).
    StructField("day", IntegerType(), False),  # O dia do mês em que o voo ocorreu (1 a 31).
    StructField("dep_time", StringType(), True),  # Horário local real de partida (hhmm).
    StructField("sched_dep_time", StringType(), True),  # Horário local programado de partida (hhmm).
    StructField("dep_delay", DoubleType(), True),  # Diferença entre os horários real e programado de partida, em minutos.
    StructField("arr_time", StringType(), True),  # Horário local real de chegada (hhmm).
    StructField("sched_arr_time", StringType(), True),  # Horário local programado de chegada (hhmm).
    StructField("arr_delay", DoubleType(), True),  # Diferença entre os horários real e programado de chegada, em minutos.
    StructField("carrier", StringType(), True),  # Código de duas letras da companhia aérea.
    StructField("flight", StringType(), True),  # Número do voo.
    StructField("tailnum", StringType(), True),  # Identificador único da aeronave.
    StructField("origin", StringType(), True),  # Código de três letras do aeroporto de origem.
    StructField("dest", StringType(), True),  # Código de três letras do aeroporto de destino.
    StructField("air_time", DoubleType(), True),  # Duração do voo, em minutos.
    StructField("distance", DoubleType(), True),  # Distância entre aeroportos de origem e destino, em milhas.
    StructField("hour", IntegerType(), True),  # Componente da hora do horário programado de partida.
    StructField("minute", IntegerType(), True),  # Componente dos minutos do horário programado de partida.
    StructField("time_hour", TimestampType(), True),  # Horário programado de partida no formato local e de data-hora.
    StructField("name", StringType(), True)  # Nome da companhia aérea do voo.
])

In [5]:
airports_db = spark.read.options(header=True).csv('./airports-database.csv', schema=schema).cache()

In [6]:
airports_db.count()

336776

In [7]:
def weekday_str(col):
  """ Converte o número do dia da semana para a string correspondente. """

  return ( F.when(col == 1, F.lit("Domingo"))
    .when(col == 2, F.lit("Segunda"))
    .when(col == 3, F.lit("Terça"))
    .when(col == 4, F.lit("Quarta"))
    .when(col == 5, F.lit("Quinta"))
    .when(col == 6, F.lit("Sexta"))
    .otherwise(F.lit("Sábado"))
  )

---

## **Perguntas**

- [1. Qual é o número total de voos no conjunto de dados?](#q1)
- [2. Quantos voos foram cancelados? (Considerando que voos cancelados têm dep_time e arr_time nulos)](#q2)
- [3. Qual é o atraso médio na partida dos voos (dep_delay)](#q3)?
- [4. Quais são os 5 aeroportos com maior número de pousos?](#q4)
- [5. Qual é a rota mais frequente (par origin-dest)?](#q5)
- [6. Quais são as 5 companhias aéreas com maior tempo médio de atraso na chegada? (Exiba também o tempo)](#q6)
- [7. Qual é o dia da semana com maior número de voos?](#q7)
- [8. Qual o percentual mensal dos voos tiveram atraso na partida superior a 30 minutos?](#q8)
- [9. Qual a origem mais comum para voos que pousaram em Seattle (SEA)?](#q19)
- [10. Qual é a média de atraso na partida dos voos (dep_delay) para cada dia da
semana?](#q10)
- [11. Qual é a rota que teve o maior tempo de voo médio (air_time)?](#q11)
- [12. Para cada aeroporto de origem, qual é o aeroporto de destino mais comum?](#q12)
- [13. Quais são as 3 rotas que tiveram a maior variação no tempo médio de voo
(air_time)?](#q13)
- [14. Qual é a média de atraso na chegada para voos que tiveram atraso na partida superior a 1 hora?](#q14)
- [15. Qual é a média de voos diários para cada mês do ano?](#q15)
- [16. Quais são as 3 rotas mais comuns que tiveram atrasos na chegada superiores a 30 minutos?](#q16)
- [17. Para cada origem, qual o principal destino?](#q17)

---

### <a class="anchor" name="q1"> 1. Qual é o número total de voos no conjunto de dados?</a>

In [8]:
(
    airports_db
    .distinct()
    .count()
)

336776

---
O conjunto de dados tem 336776 registros de vôos.

---

### <a class="anchor" name="q2">2. Quantos voos foram cancelados? (Considerando que voos cancelados têm dep_time e arr_time nulos)</a>

In [9]:
(
    airports_db
    .where((F.col('dep_time').isNull()) & (F.col('arr_time').isNull()))
    .count()
)

8255

---
Foram cancelados 8255 vôos.

---

### <a class="anchor" name="q3">3. Qual é o atraso médio na partida dos voos (dep_delay) ?</a>

In [10]:
(
    airports_db
    .where(F.col('dep_delay') > 0)
    .select(F.avg('dep_delay').alias('avg_dep_delay'))
    .show()
)

+-----------------+
|    avg_dep_delay|
+-----------------+
|39.37323252771895|
+-----------------+



In [11]:
(
    airports_db
    .select(F.avg('dep_delay').alias('avg_dep_delay'))
    .show()
)

+------------------+
|     avg_dep_delay|
+------------------+
|12.639070257304708|
+------------------+



---
Entre os vôos que de fato atrasaram (dep_delay > 0), o atraso médio é de 39.37 minutos.

Entre todos os vôos, atrasados ou não, a diferença média entre a hora agendada e a hora de partida é de aproximadamente +12.64 minutos.

---

### <a class="anchor" name="q4">4. Quais são os 5 aeroportos com maior número de pousos?</a>

In [12]:
(
    airports_db
    .groupBy('dest')
    .count()
    .orderBy("count", ascending=False)
    .limit(5)
    .show()
)

+----+-----+
|dest|count|
+----+-----+
| ORD|17283|
| ATL|17215|
| LAX|16174|
| BOS|15508|
| MCO|14082|
+----+-----+



---
1. ORD - O'Hare International Airport (Chicago, Illinois)

2. ATL - Hartsfield-Jackson Atlanta International Airport (Atlanta, Georgia)

3. LAX - Los Angeles International Airport (Los Angeles, California)

4. BOS - Logan International Airport (Boston, Massachusetts)

5. MCO - Orlando International Airport (Orlando, Florida)

---

### <a class="anchor" name="q5">5. Qual é a rota mais frequente (par origin-dest)?</a>

In [13]:
(
    airports_db.
    groupBy('origin', 'dest')
    .count()
    .orderBy("count", ascending=False)
    .limit(1)
    .show()
)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|11262|
+------+----+-----+



---
A rota mais frequente no conjunto de dados é JFK -> LAX (Nova Iorque -> Los Angeles)

---

### <a class="anchor" name="q6">6. Quais são as 5 companhias aéreas com maior tempo médio de atraso na chegada? (Exiba também o tempo)</a>

In [14]:
(
    airports_db
    .groupBy('name')
    .agg(F.avg('arr_delay')
    .alias('avg_delay'))
    .orderBy("avg_delay", ascending=False)
    .limit(5)
    .show(truncate=False)
)

+---------------------------+------------------+
|name                       |avg_delay         |
+---------------------------+------------------+
|Frontier Airlines Inc.     |21.920704845814978|
|AirTran Airways Corporation|20.115905511811025|
|ExpressJet Airlines Inc.   |15.79643108710965 |
|Mesa Airlines Inc.         |15.556985294117647|
|SkyWest Airlines Inc.      |11.931034482758621|
+---------------------------+------------------+



---

### <a class="anchor" name="q7">7. Qual é o dia da semana com maior número de voos?</a>

In [15]:
(
    airports_db
    .groupBy(F.dayofweek('time_hour'))
    .count()
    .orderBy("count", ascending=False)
    .withColumn('weekday', weekday_str(F.col('dayofweek(time_hour)')))
    .select('weekday', 'count')
    .show()
)

+-------+-----+
|weekday|count|
+-------+-----+
|Segunda|50690|
|  Terça|50422|
|  Sexta|50308|
| Quinta|50219|
| Quarta|50060|
|Domingo|46357|
| Sábado|38720|
+-------+-----+



---
O dia da semana com mais vôos é segunda-feira (50690 vôos).

---

### <a class="anchor" name="q8">8. Qual o percentual mensal dos voos tiveram atraso na partida superior a 30 minutos?</a>

In [16]:
(
    airports_db
    .groupBy('month')
    .agg((F.count(F.when(F.col('dep_delay') > 30, 1)) / F.count('*')).alias('percent_delay'))
    .sort('month')
    .show()
)

+-----+-------------------+
|month|      percent_delay|
+-----+-------------------+
|    1|0.12405569545252555|
|    2|0.12752995871908943|
|    3| 0.1494416314073663|
|    4|0.15993646311330745|
|    5|0.15335463258785942|
|    6|0.20242183903976207|
|    7|0.20978759558198812|
|    8|0.14450847342039758|
|    9| 0.0877275694494814|
|   10|0.09335733324102599|
|   11|0.08757517969781428|
|   12| 0.1731295539363782|
+-----+-------------------+



---

### <a class="anchor" name="q9">9. Qual a origem mais comum para voos que pousaram em Seattle (SEA)?</a>

In [17]:
(
    airports_db
    .where((F.col('dest') == 'SEA') & ~F.col('arr_time').isNull())
    .groupby('origin')
    .count()
    .sort('count', ascending=False)
    .limit(1)
    .show()
)

+------+-----+
|origin|count|
+------+-----+
|   JFK| 2079|
+------+-----+



---
A origem mais comum é o aeroporto JFK (John F. Kennedy International Airport, New York).

---

### <a class="anchor" name="q10">10. Qual é a média de atraso na partida dos voos (dep_delay) para cada dia da semana?</a>

In [18]:
(
    airports_db
    .groupBy(F.dayofweek('time_hour').alias('dayofweek'))
    .agg(F.avg('dep_delay').alias('avg_dep_delay'))
    .sort('dayofweek')
    .withColumn('weekday', weekday_str(F.col('dayofweek')))
    .select('weekday', 'avg_dep_delay')
    .show()
)

+-------+------------------+
|weekday|     avg_dep_delay|
+-------+------------------+
|Domingo|11.589531801152422|
|Segunda|14.778936729330908|
|  Terça|10.631682565455652|
| Quarta|11.803512219083876|
| Quinta|16.148919990957108|
|  Sexta| 14.69605749486653|
| Sábado| 7.650502333676133|
+-------+------------------+



---

### <a class="anchor" name="q11">11. Qual é a rota que teve o maior tempo de voo médio (air_time) ?</a>

In [19]:
(
    airports_db
    .groupBy('origin', 'dest')
    .agg(F.avg('air_time').alias('avg_air_time'))
    .orderBy('avg_air_time', ascending=False)
    .limit(1)
    .show()
)

+------+----+-----------------+
|origin|dest|     avg_air_time|
+------+----+-----------------+
|   JFK| HNL|623.0877192982456|
+------+----+-----------------+



---
A rota com maior tempo de vôo médio foi JFK -> HNL (Nova Iorque -> Honolulu)

---

### <a class="anchor" name="q12">12. Para cada aeroporto de origem, qual é o aeroporto de destino mais comum?</a>

In [20]:
(
    airports_db
    .groupBy('origin', 'dest')
    .count()
    .withColumn('rank', F.rank().over(Window.partitionBy("origin").orderBy(F.col('count').desc())))
    .where(F.col('rank') == 1)
    .select('origin', 'dest', 'count')
    .show()
)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   EWR| ORD| 6100|
|   JFK| LAX|11262|
|   LGA| ATL|10263|
+------+----+-----+



---
JFK -> LAX (11262 vôos)

LGA -> ATL (10263 vôos)

EWR -> ORD (6100 vôos)

---

### <a class="anchor" name="q13">13. Quais são as 3 rotas que tiveram a maior variação no tempo médio de voo (air_time) ?<a/>

In [21]:
(
    airports_db
    .groupBy('origin', 'dest')
    .agg(F.stddev('air_time')
    .alias('stddev_air_time'))
    .orderBy('stddev_air_time', ascending=False)
    .limit(3)
    .show()
)

+------+----+------------------+
|origin|dest|   stddev_air_time|
+------+----+------------------+
|   LGA| MYR| 25.32455988429677|
|   EWR| HNL| 21.26613546847427|
|   JFK| HNL|20.688824842787056|
+------+----+------------------+



---
As 3 rotas com maior variação no tempo médio de vôo, de acordo com o desvio-padrão, foram:

LGA -> MYR

EWR -> HNL

JFK -> HNL

---

### <a class="anchor" name="q14">14. Qual é a média de atraso na chegada para voos que tiveram atraso na partida superior a 1 hora<a/>

In [22]:
(
    airports_db
    .where(F.col('dep_delay') > 60)
    .select(F.avg('arr_delay')
    .alias('avg_delay'))
    .show()
)

+------------------+
|         avg_delay|
+------------------+
|119.04880549963919|
+------------------+



---

### <a class="anchor" name="q15">15. Qual é a média de voos diários para cada mês do ano?<a/>

In [23]:
(
    airports_db
    .groupby('day', 'month')
    .count()
    .groupby('month').agg(F.avg('count').alias('daily_avg'))
    .sort('month')
    .show()
)

+-----+-----------------+
|month|        daily_avg|
+-----+-----------------+
|    1|871.0967741935484|
|    2|891.1071428571429|
|    3|930.1290322580645|
|    4|944.3333333333334|
|    5|928.9032258064516|
|    6|941.4333333333333|
|    7|949.1935483870968|
|    8|946.0322580645161|
|    9|919.1333333333333|
|   10|931.9032258064516|
|   11|908.9333333333333|
|   12|907.5806451612904|
+-----+-----------------+



---

### <a class="anchor" name="q16">16. Quais são as 3 rotas mais comuns que tiveram atrasos na chegada superiores a 30 minutos?<a/>

In [24]:
(
    airports_db
    .where(F.col('arr_delay') > 30)
    .groupBy('origin', 'dest')
    .count()
    .orderBy('count', ascending=False)
    .limit(3)
    .show()
)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   LGA| ATL| 1563|
|   JFK| LAX| 1286|
|   LGA| ORD| 1188|
+------+----+-----+



---

### <a class="anchor" name="q17">17. Para cada origem, qual o principal destino?</a>

In [25]:
(
    airports_db
    .groupBy('origin', 'dest')
    .count()
    .withColumn('rank', F.rank().over(Window.partitionBy("origin").orderBy(F.col('count').desc())))
    .where(F.col('rank') == 1)
    .select('origin', 'dest', 'count')
    .show()
)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   EWR| ORD| 6100|
|   JFK| LAX|11262|
|   LGA| ATL|10263|
+------+----+-----+



---
Esta questão parece ser semelhante à [questão 12](#q12).

---

## **Enriquecimento da Base de Dados**

---

### **Consolidar coordenadas dos aeroportos em um csv**

In [26]:
import time
import pytz
import requests
import csv

def get_coordinates(airport):
  time.sleep(1)

  airportdb_key = userdata.get('AIRPORTDB_KEY')
  url = f"https://airportdb.io/api/v1/airport/K{airport}?apiToken={airportdb_key}"
  response = requests.get(url)
  data = response.json()

  if data.get('statusCode') == 404:
    return None, None

  lat, long = data['latitude_deg'], data['longitude_deg']
  return lat, long


distinct_airports = (
    airports_db
    .select(F.col("dest").alias('airport'))
    .distinct()
    .union(
        airports_db
        .select(F.col("origin").alias('airport'))
        .distinct()
    )
)
airport_codes = [row.airport for row in distinct_airports.collect()]

# Incluindo manualmente informações sobre aeroportos que não constam na api
missing_data = '''
PSE,18.0083,-66.5630
HNL,21.3187,-157.9225
SJU,18.4394,-66.0018
BQN,18.4949,-67.1294
STT,18.3373,-64.9734
ANC,61.1743,-149.9984
'''

with open('coordinates.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(['airport', 'lat', 'long'])
  for airport in airport_codes:
    lat, long = get_coordinates(airport)
    if lat and long:
      writer.writerow([airport, lat, long])
  f.write(missing_data.strip())


---

### **Enriquecer dados com respectivas coordenadas de origem e destino**

In [28]:
coordinates_schema = StructType([
    StructField("airport", StringType(), False),
    StructField("lat", DoubleType(), False),
    StructField("long", DoubleType(), False),

])

coordinates = spark.read.options(header=True).csv('./coordinates.csv', schema=coordinates_schema).distinct()

airports_db = (
    airports_db
    .join(coordinates, F.col('origin') == F.col('airport'), 'left_outer')
    .withColumnRenamed('lat', 'lat_origin')
    .withColumnRenamed('long', 'long_origin')
    .drop('airport')
    .join(coordinates, F.col('dest') == F.col('airport'), 'left_outer')
    .withColumnRenamed('lat', 'lat_dest')
    .withColumnRenamed('long', 'long_dest')
    .withColumnRenamed('tz_offset', 'tz_offset_dest')
    .drop('airport')
    .cache()
)

---

### **Consolidar a velocidade do vento na origem de todos os registros em um csv**

In [30]:
import json
from datetime import datetime, timedelta

def get_wind_speed(year, month, day, lat, lon, apikey):
    try:
        url = f'https://api.weatherbit.io/v2.0/history/daily'
        start_date = datetime(int(year), int(month), int(day))
        end_date = start_date + timedelta(days=1)

        params = {
            'lat': lat,
            'lon': lon,
            'start_date': start_date.strftime('%Y-%m-%d') ,
            'end_date': end_date.strftime('%Y-%m-%d'),
            'key': apikey
        }
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = json.loads(response.text)
        wind_spd = data['data'][0]['wind_spd']
        return float(wind_spd)
    except (requests.exceptions.RequestException, KeyError, IndexError, json.JSONDecodeError) as e:
        print(f"Error fetching weather data: {e}")
        return None

origin_days = airports_db.select('origin', 'month', 'day', 'lat_origin', 'long_origin').distinct().collect()

with open('windspeeds.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(['airport', 'month', 'day', 'wind_spd'])
  for row in origin_days:
    airport, month, day, lat, long = row
    wind_spd = get_wind_speed(2013, month, day, lat, long, apikey=userdata.get('WEATHERBIT_KEY'))
    time.sleep(1)
    if wind_spd:
      writer.writerow([airport, month, day, wind_spd])

---

### **Enriquecer dados com velocidade do vento na origem**

In [32]:
windspeeds_schema = StructType([
    StructField("airport", StringType(), False),
    StructField("wind_month", IntegerType(), False),
    StructField("wind_day", IntegerType(), False),
    StructField("wind_spd", DoubleType(), False)

])

windspeeds = spark.read.options(header=True).csv('./windspeeds.csv', schema=windspeeds_schema)

airports_db = (
  airports_db.alias('a')
  .join(
      windspeeds.alias('w'),
      (F.col('origin') == F.col('airport'))
        & (F.col('month') == F.col('wind_month'))
        & (F.col('day') == F.col('wind_day')),
      'left_outer', )
  .withColumnRenamed('wind_spd', 'origin_wind_spd')
  .drop('airport', 'wind_month', 'wind_day')
)

---

### **Pergunta final**

Enriqueça a base de dados de voos com as condições meteorológicas (velocidade do vento) para os aeroportos de origem e destino. Mostre as informações enriquecidas para os 5 voos com maior atraso na chegada.

*Observação: como existe um limite de requisições diárias na api do weatherbit (1.5K req/dia), optou-se por enriquecer a totalidade dos dados relativos à origem, dado que é um número reduzido de combinações, enquanto para os dados relativos ao destino, o enriquecimento foi realizado apenas para os 5 voos com maior atraso na chegada, que era o objeto da questão.*

In [ ]:
@F.udf(returnType=DoubleType())
def wind_speed_udf(year, month, day, lat, lon, apikey=userdata.get('WEATHERBIT_KEY')):
  return get_wind_speed(year, month, day, lat, lon, apikey)

In [34]:
(
  airports_db
  .sort('arr_delay', ascending=False)
  .limit(5)
  .withColumn('dest_wind_spd', wind_speed_udf(F.col('year'), F.col('month'), F.col('day'), F.col('lat_dest'), F.col('long_dest')))
  .show()
)

+------+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+----------+-----------+-----------------+----------+---------------+-------------+
|    id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|lat_origin|long_origin|         lat_dest| long_dest|origin_wind_spd|dest_wind_spd|
+------+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+----------+-----------+-----------------+----------+---------------+-------------+
|  7072|2013|    1|  9|   641.0|           900|   1301.0|  1242.0|          1530|   1272.0|     HA|    51| N384HA|   JFK| HNL|   640.0|  4983.0|   9|     0|2013

---

## **Modelo de ML**

---

### **Pré-processamento**

In [119]:
import calendar
import numpy as np
from sklearn.model_selection import train_test_split
from itertools import chain

days_in_month = {k:calendar.monthrange(2013, k)[1] for k in range(1, 13)}
days_in_month_mapping = F.create_map([F.lit(x) for x in chain(*days_in_month.items())])

df = (
    airports_db.select(
      'month',
      'day',
      'hour',
      (F.col('day') / days_in_month_mapping[F.col('month')]).alias('scaled_day'),
      F.dayofweek('time_hour').alias('weekday'),
      F.col('carrier'),
      F.col('distance'),
      F.col('origin_wind_spd').alias('wind_spd'),
      F.col('arr_delay')
    )
    .where(F.col('arr_delay').isNotNull())
    .toPandas()
)

X = df[['month', 'day', 'scaled_day', 'hour', 'weekday', 'carrier', 'distance', 'wind_spd']]
y = df['arr_delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=42)

---

### **Engenharia de features**

In [40]:
from sklearn.base import BaseEstimator, TransformerMixin


class CosineEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, max_val):
        self.max_val = max_val

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_rad = X / self.max_val * 2 * np.pi
        return np.cos(X_rad)


In [120]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('month_cosine_encoder', CosineEncoder(max_val=12), ['month']),
        ('day_cosine_encoder', CosineEncoder(max_val=1), ['scaled_day']),
        ('hour_cosine_encoder', CosineEncoder(max_val=24), ['hour']),
        ('weekday_cosine_encoder', CosineEncoder(max_val=7), ['weekday']),
        ('month_onehot_encoder', OneHotEncoder(handle_unknown="ignore", sparse_output=False), ['month']),
        ('day_onehot_encoder', OneHotEncoder(handle_unknown="ignore", sparse_output=False), ['day']),
        ('hour_onehot_encoder', OneHotEncoder(handle_unknown="ignore", sparse_output=False), ['hour']),
        ('weekday_onehot_encoder', OneHotEncoder(handle_unknown="ignore", sparse_output=False), ['weekday']),
        ('carrier_onehot_encoder', OneHotEncoder(handle_unknown="ignore", sparse_output=False), ['carrier']),
        ('distance_minmax_encoder', MinMaxScaler(), ['distance']),
        ('origin_wind_spd_minmax_encoder', MinMaxScaler(), ['wind_spd'])
    ]
)

---

### **Treinamento do modelo + Ajuste de Hyperparâmetros**

In [125]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import math

linear_regression_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SGDRegressor())
])

xgboost_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

hyperparameters = {
    'linear_regression': {
        'regressor__eta0': [0.1, 0.01, 0.001],
        'regressor__alpha': [0.1, 0.01, 0.001]
    },
    'xgboost': {
        'regressor__n_estimators': [300, 400, 500],
        'regressor__max_depth': [7, 8, 9, 10],
        'regressor__min_child_weight': [1, 2, 3, 4],
        'regressor__eta': [0.3, 0.1, 0.03, 0.01]
    }
}


# Train and evaluate models
results = {}
best_mse = math.inf
best_model = {}
best_pipeline = None

for model_name, pipeline in [('linear_regression', linear_regression_pipeline), ('xgboost', xgboost_pipeline)]:
    results[model_name] = {}
    for i, hyperparameter_setting in enumerate(ParameterGrid(hyperparameters[model_name])):
        pipeline.set_params(**hyperparameter_setting)
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)
        val_mse = mean_squared_error(y_val, y_pred)
        train_mse = mean_squared_error(y_train, pipeline.predict(X_train))

        r2 = r2_score(y_val, y_pred)
        results[model_name][str(hyperparameter_setting)] = {'mse': val_mse, 'r2': r2}
        if val_mse < best_mse:
          best_mse = val_mse
          best_model = {
            'val_mse': val_mse,
            'train_mse': train_mse,
            'r2': r2,
            'model': model_name,
            'params': hyperparameter_setting
          }
          best_pipeline = pipeline
        print(f'model={model_name} | val_mse={val_mse} | train_mse={train_mse} | r²={r2} | params={hyperparameter_setting}')


with open(f'best_model.pkl', 'wb') as f:
  pickle.dump(best_pipeline, f)

print("="*80)

print(f"Best model: {best_model['model']}")
print(f"Best hyperparameters: {best_model['params']}")
print(f"Val MSE: {best_model['val_mse']:.4f}")
print(f"Train MSE: {best_model['train_mse']:.4f}")
print(f"R^2: {best_model['r2']:.4f}")

model=linear_regression | val_mse=1836.0648434126583 | train_mse=1892.3152697669436 | r²=0.0552175913100762 | params={'regressor__alpha': 0.1, 'regressor__eta0': 0.1}
model=linear_regression | val_mse=1822.9798146909118 | train_mse=1876.290332125478 | r²=0.06195074400773948 | params={'regressor__alpha': 0.1, 'regressor__eta0': 0.01}
model=linear_regression | val_mse=1820.5009251527613 | train_mse=1874.2687215549029 | r²=0.06322630420221342 | params={'regressor__alpha': 0.1, 'regressor__eta0': 0.001}
model=linear_regression | val_mse=1775.5849304466822 | train_mse=1830.0880165751057 | r²=0.08633869144679385 | params={'regressor__alpha': 0.01, 'regressor__eta0': 0.1}
model=linear_regression | val_mse=1769.6205165715132 | train_mse=1821.013498917082 | r²=0.08940779509398955 | params={'regressor__alpha': 0.01, 'regressor__eta0': 0.01}
model=linear_regression | val_mse=1767.3238644296575 | train_mse=1819.7838255210954 | r²=0.09058958153812857 | params={'regressor__alpha': 0.01, 'regressor__

---

### **Avaliação no conjunto de testes**

In [127]:
y_pred = best_pipeline.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
r2_test = r2_score(y_test, y_pred)

print(f"Test MSE: {test_mse:.4f}")
print(f"R^2: {r2_test:.4f}")

Test MSE: 1450.3990
R^2: 0.2714
